<a href="https://colab.research.google.com/github/raulpoppiel/geemap/blob/master/01_junior/01_Zonal_statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://media.licdn.com/dms/image/D4D0BAQHwOVv97dZgRA/company-logo_200_200/0/1694021955341/neutral_farming_logo?e=1715212800&v=beta&t=dku1R4nBzkf1VXEv_yofHmfrzOMDKEPXQ24w7EfLUY8" height="150"></center>

# "Compute zonal statistics by polygon"

_Tool created by **Raul Roberto Poppiel**_: [raul@neutralfarming.earth](raul@neutralfarming.earth)

This tool computes statistical values (e.g., mean) within each geometry polygon from rasters (e.g., soil maps) using Google Earth Engine (GEE)

General workflow:
1. Import region of interest (shp, polygons) from GEE
2. Import soil maps from GEE
3. Compute zonal statistics from maps
4. Plot results

GEE Code Editor: https://code.earthengine.google.com/

## Install and import modules

Import the required modules

In [ ]:
# Load modules
import os
import ee, geemap
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import time
from pathlib import Path
from datetime import datetime

Authenticate and Initialize Earth Engine and geemap.

In [ ]:
# Run and then paste the GEE token
#geemap.ee_initialize(project='apps-388723')
ee.Authenticate()
ee.Initialize(project='apps-388723')

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/cloud-platform%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=7NFSKNkiohi9IsJVuOJKhkVSdA3uFnzs3ZR0d6VKrQU&tc=hVwzktk7vXvNakTkb7TYlm8LrMX8r5HaVbrllvbT-Yo&cc=VpumabQ4txncIJAuWvCO41nfrS7wg3vRe1sKHUiL4zs

The authorization workflow will generate a code, which you should paste in the box below.


KeyboardInterrupt: Interrupted by user

## Set Google Drive

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Authenticate Google Cloud Platform

In [ ]:
import io
import google.cloud.storage

from google.colab import auth

PROJECT_ID = "apps-388723"
auth.authenticate_user(project_id=PROJECT_ID)

Define the output folders in GD

In [ ]:
# Check vector "file_name" available in Assets from Raul GEE account
print('Vector files within Raul´s PROJECTS directory in GEE Assets \n')
src_folder = "projects/ee-raul/assets/Projects"
assets = ee.data.listAssets({'parent': src_folder})['assets']

# ANSI color codes
COLOR_RED = '\033[91m'
COLOR_BLUE = '\033[94m'
COLOR_RESET = '\033[0m'

# Initialize variables
prev_prefix = None
is_blue = True

# Loop through assets and print
for asset in assets:
    last_element = asset['id'].split("/")[-1]
    prefix = last_element[:4]

    if prefix != prev_prefix:
        print(f"{COLOR_BLUE}{'_' * len(last_element)}{COLOR_RESET}")
        is_blue = not is_blue
    print(f"{COLOR_BLUE if is_blue else COLOR_RED}{last_element}{COLOR_RESET}")

    prev_prefix = prefix

In [ ]:
# Define folder names
folder_name_root = 'Projects'                   # Do not change it

folder_name_project = 'Rosario'             # PROYECTO: ej. SanEnrique,  SanJoseFarms, ConchayToro, VinaMontes
folder_name_farm = 'Cerro'                 # FUNDO: ej. QuebradaSeca, ElTriangulo, LaRosa

folder_name_product = 'Junior'                  # Do not change it
folder_name_specific = '01_Zonal_statistics'    # Do not change it

# Insert the vector file name from GEE Assets (files will be exported using this name)
file_name = 'Rosario_Cerro'                    # EL NOMBRE DEL SHAPEFILE ALMACENADO EN ASSETS DE GEE de Raul (listado arriba)

Folder Structure:
```
Google Drive
└── folder_name_root
      └── folder_name_project
          └── folder_name_farm
              └── folder_name_specific
                   └── 📈 results..{file_name}...
```

In [ ]:
# Check if the folder exists or else create
root_path = f'/content/drive/MyDrive/{folder_name_root}/{folder_name_project}'
out_path = f'{root_path}/{folder_name_farm}/{folder_name_product}/{folder_name_specific}' # your results will be stored in 'out_path'

if not os.path.exists(out_path):
  Path(out_path).mkdir(parents=True, exist_ok=True)
  print("Output directory created successfully. \n")
  os.chdir(out_path)
  print(os.getcwd(),'\n')  # Print the current working directory

else:
  print("Output directory already exists.")
  os.chdir(out_path)
  print(os.getcwd(),'\n')  # Print the current working directory
  display(pd.DataFrame(os.listdir(),columns=['List files']))  # List files and directories in the current directory

## Start processing

### Feature collection

Import the features from your Assets

In [ ]:
# Import farm plot boundaries (shapefiles)

fc = ee.FeatureCollection(f'projects/apps-388723/assets/{file_name}')

print('FC size: ',fc.size().getInfo(),'\n')

# Display the table of attributes for one feature/polygon
print('FC attribute table (1st feature): \n')
dic = fc.limit(1).getInfo()['features'][0]['properties']
pd.DataFrame(dic.values(),dic.keys()).T

In [ ]:
# Select an attribute name (column) that contains the ID of polygons
ID = 'Name'

fc_to_reduction = fc.select([ID],["Name"])

fc

Export the original vector to DG

In [ ]:
# Create a folder to export input vector to GDrive
folder_vector_specific = f'00_input_vector_{folder_name_farm}'
input_vector_path = f'/content/drive/MyDrive/{folder_name_root}/{folder_name_project}/{folder_name_farm}/{folder_vector_specific}'

if not os.path.exists(input_vector_path):
    Path(input_vector_path).mkdir(parents=True, exist_ok=True)
    print("Output directory created successfully.")
else:
    print("Output directory already exists.")
    os.chdir(input_vector_path)
    print(os.getcwd(),'\n')  # Print the current working directory
    display(pd.DataFrame(os.listdir(),columns=['List files']))  # List files and directories in the current directory

# Check if file_name already exists
if f'{file_name}.shp' in os.listdir():
    print(f"\n A file with the name '{file_name}.shp' already exists in the directory.")
else:
    # Proceed with ee_export_vector_to_drive if file exists
    print('\n')
    geemap.ee_export_vector_to_drive(fc, description=file_name, folder=folder_vector_specific, fileFormat='SHP')

### Importing raster data

Importing soil maps at 90m spatial resolution

In [ ]:
# Import soil maps (0-20cm depth) from Latin America and the Caribbean (LAC) region
c     = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac/assets/DSM-LAC/C_gkg_PredictedImages').mosaic().select(1).rename('C_gkg')
cstk  = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac2/assets/Cstock_tha_PredictedImages').mosaic().select(1).rename('CsACTtha')
clay  = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac/assets/DSM-LAC/Clay_gkg_PredictedImages').mosaic().select(1).rename('Clay_gkg')
silt  = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac/assets/DSM-LAC/Silt_gkg_PredictedImages').mosaic().select(1).rename('Silt_gkg')
sand  = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac/assets/DSM-LAC/Sand_gkg_PredictedImages').mosaic().select(1).rename('Sand_gkg')
bd    = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac-3/assets/Density_kgdm3_PredictedImages').mosaic().select(1).rename('BD_kgdm3')
th    = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac2/assets/Th_15atm_PredictedImages').mosaic().select(1).multiply(100).rename('PAW_perc')
cec   = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac2/assets/CEC_Ph7_mmolkg_PredictedImages').mosaic().select(1).divide(10).rename('CEC_meq100')
ph    = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac2/assets/pH_H2O_PredictedImages').mosaic().select(1).rename('pH_water')
# p     = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac-3/assets/P_mgkg_PredictedImages').mosaic().select(1).rename('P_mgkg')
# k     = ee.ImageCollection('projects/ee-raulrpoppiel-dsm-lac-4/assets/K_mmolkg_PredictedImages').mosaic().select(1).multiply(10).rename('K_mgkg')

# Import a DEM
dem = ee.Image("USGS/SRTMGL1_003").rename('Elev_m')

Compute the potencial (maimum) and deficit SOC stock

In [ ]:
# ACTUAL
# Carbon content actual (stable pool)
# Fractions: ~85% Stable pool, retained in the mineral fraction - https://doi.org/10.1016/j.geoderma.2018.07.026
#            ~15% Labile pool, retained in aggregates

# carbon_actual = ee.Image().expression(
# #    '(((0.8966 * SOCtotal) + 0.0773)', # https://doi.org/10.1016/j.geoderma.2023.116549
#     '0.8 * SOCtotal',
#      {'SOCtotal': c.select('C_gkg')}).rename('c_actual_gkg')

# Carbon stock actual (stable pool)
carbon_stock_actual = cstk.rename('c_stk_actual_tha')
# carbon_stock_actual = ee.Image().expression(
#     '(c_actual * BD * SoilThickness_cm) /10', # https://doi.org/10.1016/j.geoderma.2023.116549
#      {'c_actual': carbon_actual.select('c_actual_gkg'),'BD': bd.select('BD_kgdm3'),'SoilThickness_cm': ee.Image(20)}).rename('c_stk_actual_tha')



# POTENTIAL
# Carbon content saturation potential (maximum SOC content)

# # Feller and Beare (1997) for tropical soils: https://doi.org/10.1016/S0016-7061(97)00039-6
# carbon_potential = ee.Image().expression(
#     '0.32 + 0.029 * (Clay + Silt)',
#      {'Clay': clay.select('Clay_gkg'),'Silt': silt.select('Silt_gkg')}).rename('c_potential_gkg')

# Bayer (2022): https://doi.org/10.1016/j.geoderma.2022.115711
carbon_potential = ee.Image().expression(
    '0.082 * ((Clay + Silt))',
     {'Clay': clay.select('Clay_gkg'),'Silt': silt.select('Silt_gkg')}).rename('c_potential_gkg')



# DEFICIT
# Carbon content saturation deficit (Δ = potential - actual)
# carbon_deficit = ee.Image().expression(
#     'carbon_potential - ((0.8966 * SOCtotal) + 0.0773)',
#     {'carbon_potential': carbon_potential.select('c_potential'), 'SOCtotal': c.select('C_gkg')}
# ).rename('c_deficit')

# Carbon stock saturation deficit (Δ = potential - actual)
carbon_stock_deficit = ee.Image().expression(
    '((Cpot * BD * SoilThickness_cm) /10) - Cstk_actual',
    {'Cpot': carbon_potential.select('c_potential_gkg'),'BD': bd.select('BD_kgdm3'),'SoilThickness_cm': ee.Image(20), 'Cstk_actual': carbon_stock_actual.select('c_stk_actual_tha')}
).rename('CsDEFtha')

# Remap negative values to zero
carbon_stock_deficit = carbon_stock_deficit.where(carbon_stock_deficit.lt(0), 0)



# POTENTIAL
# Carbon stock saturation potential (maximum SOC stock)
carbon_stock_potential = carbon_stock_actual.add(carbon_stock_deficit).rename('CsPOTtha')

In [ ]:
# Select the maps for assessment
maps = [c, cstk, clay, silt, sand, bd, th, cec, ph, carbon_stock_potential, carbon_stock_deficit, dem]
band_list = ee.Image(maps).bandNames().getInfo()

pd.DataFrame(band_list, columns=['Band names'])

### Compute mean values by polygon and export as vector

In [ ]:
# Define the statistic
# Statistics: "MEAN", "MAXIMUM", "MEDIAN","MINIMUM","MODE","STD","MIN_MAX","SUM","VARIANCE", "COUNT"
statistics_Type = 'MEAN'

# Define static date of compute, for now when the script runs
simplified_date = datetime.now().strftime('%Y-%m-%d')

# Define the output path to save results
global_stats_path = f'{out_path}/{file_name}_zonal_statistics_{statistics_Type}_{simplified_date}'

print(global_stats_path)

In [ ]:
# Reducing daily climatic values by zone
scale = 90.0

# Maps to reduce by polygon
img = ee.ImageCollection(maps)

parameters = {
    'in_value_raster': img,
    'statistics_type':statistics_Type,
    'scale':float(scale), # a high scale avoid missing values
    'tile_scale':16.0,
    'return_fc':False,
    'timeout':3000,
    }

In [ ]:
geemap.zonal_statistics(in_zone_vector=fc_to_reduction,out_file_path=f'{global_stats_path}.shp',**parameters)

### Add base columns to csv stat

In [ ]:
# Import stats by zone and verify values
zonal_stat = gpd.read_file(f'{global_stats_path}.shp', driver='ESRI Shapefile', encoding='latin1') # utf-8
zonal_stat.head(1)

In [ ]:
# Remove the last 3 characters from each string in list_of_headers
list_of_headers = [header[:-2] for header in band_list]

# Rename headers
for header in list_of_headers:
    matching_columns = zonal_stat.columns[zonal_stat.columns.str.contains(header)]
    for col in matching_columns:
        new_header = col.split('_', 1)[1]  # Split at first '_' and take the second part
        zonal_stat.rename(columns={col: new_header}, inplace=True)

In [ ]:
list(zonal_stat.columns)

Round to two decimal places

In [ ]:
def round_it(x):
  return round(x, 2) if x.dtype == "float64" else x

zonal_stat = zonal_stat.apply(round_it)

zonal_stat.head(2)

In [ ]:
# Create a column with predio/fundo name
zonal_stat['predio'] = str(folder_name_farm)

# Reproject the GeoDataFrame to EPSG:6933 (Equal Area Cylindrical)
zonal_stat_m = zonal_stat.to_crs(epsg=6933)

# Calculate area directly in hectares and round to two decimal places
zonal_stat['area_ha'] = (zonal_stat_m['geometry'].area / 10000).round(2)

zonal_stat.head(2)

### Export to GDrive

In [ ]:
# Save
zonal_stat.to_file(global_stats_path+'.shp', driver='ESRI Shapefile', encoding='utf-8')
zonal_stat.to_file(global_stats_path+'.geojson', driver='GeoJSON', encoding='utf-8')
zonal_stat.drop(columns='geometry').to_csv(global_stats_path+'.csv', encoding='utf-8', index=False)

print('\n',f'Files sucessfully exported to {global_stats_path}')

### Export to GCS

In [ ]:
from google.cloud import storage
bucket_name = "neutralfarming-test-files"

def camel_to_snake(camel_case_string):
   snake_case_string = ""
   for i, c in enumerate(camel_case_string):
      if i == 0:
         snake_case_string += c.lower()
      elif c.isupper():
         snake_case_string += "_" + c.lower()
      else:
         snake_case_string += c

   return snake_case_string

def write_to_gcs(bucket_name, blob_name):
    """Write and read a blob from GCS using file-like IO"""

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    with io.BytesIO() as buf:
      zonal_stat.to_file(buf, driver='GeoJSON')
      byte_str = buf.getvalue()
      str_buf = byte_str.decode('UTF-8')

      with blob.open("w", content_type='application/json') as f:
          f.write(str_buf)

    blob.make_public()

write_to_gcs(bucket_name, f'{camel_to_snake(folder_name_project)}/{camel_to_snake(folder_name_farm)}/{file_name}_zonal_statistics_{statistics_Type}_{simplified_date}.geojson')